In [2]:
import numpy as np
import pandas as pd
import sklearn
import math 
from typing import AnyStr, Callable, Tuple
from sklearn.base import ClusterMixin,BaseEstimator
from scipy.cluster.hierarchy import ward, cut_tree
from sklearn.metrics import make_scorer
from sklearn.cluster import AffinityPropagation
from sklearn.decomposition import PCA
import ClusterEnsembles as CE

from load_data import *
from pred_score import *
from Filter_FS import *
from hybrid_FS import *
from overlap_genes import *
from crossValidation import *

# Iterative clustering

In [82]:
class IterativeClustering(ClusterMixin, BaseEstimator):
    '''
    Iterative Families clustering
    Hierachical clustering with the ward2 criterion, use the pearson's correlation as the distance measure.
    Parameters
    ----------
    family_interest: np.array,
        list of family of interest
    Scoring : Callable,
        scoring function use to evaluate the model
    maximize: bool,
        if True the scoring function is maximize, else it is minimize
        
    Attributes
    ----------
    n_clusters_ : int
        The number of clusters found by the algorithm.
    labels_ : ndarray of shape (n_samples)
        Cluster labels for each point.
    family_interest: np.array,
        list of family of interest.
    Scoring : Callable,
        scoring function use to evaluate the model.
    maximize: bool,
        if True the scoring function is maximize, else it is minimize.
    
    '''
    
    def __init__(self, family_interest_:np.array, Scoring_:Callable, maximize_:bool):
        super().__init__()
        self.family_interest_ = family_interest_
        self.Scoring_ = Scoring_
        self.maximize_ = maximize_
        
    def fit(self, X:np.array, y:np.array, N:int =2, iterations:int =20, subset: np.array = None):
        '''Fit data using hierachical clustering with the ward2 criterion and use the spearman correlation as the distance measure and predict.
        
        parameters:
        -------
        x : np.array,
            features of each data points
        y : np.array,
            family of each data points
        N : int,
            max number of
        iterations : int,
            number of iterative clustering
    

        Returns
        -------
        self,
            return fitted self'''
        
        #Iterative clustering algorithm
        result = iterative_clustering(X,y , N, iterations)
        r = result[0]
        rcsv = pd.DataFrame(r)
        rcsv.to_csv('../data/test.csv', index=False)
        
        
        #Score the cluster and determine the number of clusters
        TP,FP,tot = test_prediction_multiple_overlap_3(result, result, result,y)
        score = TP/(TP+FP)
        self.score_ = score
        
        return self, TP, FP  
    
    def score(self, X, y_true):
        return self.score_

In [83]:
A = [1,20,1,20,20,1,100,100,100]
B = [100,30,100,30,30,100,1,1,1]
norm_data = np.array([A,B])
y_test = [1,2,1,2,2,1,3,3,3]

In [84]:
model = IterativeClustering(np.unique(y_test),compute_precision,True)
result  = model.fit(norm_data,y_test,N=2, iterations=20)
model.score_

0.6666666666666666

In [85]:
#Load preprocess data
AE3= np.array(pd.read_csv ('../data/processed_data/AE3.csv'))
y = np.array(AE3[:,-1],dtype=int)
AE3 = AE3[:,0:-1]

In [87]:
model = IterativeClustering(np.unique(y),compute_precision,True)
result,TP,FP  = model.fit(AE3,y,N=2, iterations=20)
print(model.score_)

print(TP,FP)

0.017167381974248927
8 458


In [12]:
#Define parameters for MIM method
N = np.arange(80,1500,30)
kwargs = {'Model': IterativeClustering, 'Scoring': compute_precision,'maximize': True,'N': N, 'n_neighbors': 3, 'plot': True} 

subset, score_training, score_testing = cross_validation(y,AE3, IterativeClustering, compute_precision,True, 5,  MIM, **kwargs)

mean_score_test, std_score_test = np.mean(score_testing), np.std(score_testing)    
print('test', mean_score_test, std_score_test)

#Predict and evaluate on whole data  set
model = IterativeClustering(np.unique(y),compute_precision,True)
result  = model.fit(AE3,y,N=2, iterations=20)
model.score_


TypeError: fit() missing 1 required positional argument: 'y'

# Repeated prediction

In [ ]:
#Load preprocess data
AE3= np.array(pd.read_csv ('../data/processed_data/AE3.csv'))
y = np.array(AE3[:,-1],dtype=int)
AE3 = AE3[:,0:-1]

genes_AE3 = np.squeeze(pd.read_csv ('../data/processed_data/AE3genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE3genes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_AE3 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
AE3 = AE3[:,ind_opt_genes]
print(AE3.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(333, 1540)
1540


In [51]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = AE3, y= y)
print(model.score_, model.recovery)

0.7619047619047619 0.6756756756756757


In [52]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 1)
result  = model.fit_predict(X = AE3, y= y)
print(model.score_, model.recovery)

0.9833333333333333 0.2972972972972973


In [54]:
thereshold = np.linspace(0.1,1.0,10)

for t in thereshold:    
    model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = t)
    result  = model.fit_predict(X = AE3, y= y)
    print('thereshold: ', t, model.score_, model.recovery)

thereshold:  0.1 0.1327319587628866 0.9129129129129129
thereshold:  0.2 0.3282442748091603 0.8528528528528528
thereshold:  0.30000000000000004 0.42118863049095606 0.8228228228228228
thereshold:  0.4 0.6481481481481481 0.7567567567567568
thereshold:  0.5 0.7619047619047619 0.6756756756756757
thereshold:  0.6 0.8344370860927153 0.6186186186186187
thereshold:  0.7000000000000001 0.8518518518518519 0.5675675675675675
thereshold:  0.8 0.8869565217391304 0.5015015015015015
thereshold:  0.9 0.8775510204081632 0.45345345345345345
thereshold:  1.0 0.9833333333333333 0.2972972972972973


In [53]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE3,y)
print(model.score_, model.recovery)

0.6826923076923077 0.7807807807807807


In [95]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='cspa')
result  = model.fit_predict(X = AE3, y = y)
print(model.score_, model.recovery)

(101, 333)
0.4294478527607362 0.990990990990991


In [ ]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='hgpa')
result  = model.fit_predict(X = AE3, y = y)
print(model.score_, model.recovery)

# Using other genesets

In [10]:
#Load AE3 data
AE3 = pd.read_csv ('../data/merged_data/D0.csv')
AE3 = AE3.set_index('Unnamed: 0')
y_AE3 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_AE4.csv')))

AE3_master = pyreadr.read_r('../data/data_master/Master_Almut_AE4_20_3t5_nocellcyclesplit.rds')
AE3_master = AE3_master[None]
AE3_master = AE3_master.set_index('gene')     

CV2_of_mean = AE3_master['P_value_estimate_CV2_ofmeans_20_']
intraCV2 = AE3_master['P_value_estimate_intraCV2_20_']
corr = AE3_master['P_value_estimate_intraCV2_20_']

CV2_of_mean = CV2_of_mean[CV2_of_mean <= 0.05].index
pd.DataFrame(CV2_of_mean).to_csv('../data/optimized_subsets/AE4CV2mean.csv', index=False)
intraCV2 = intraCV2[intraCV2 <= 0.05].index
pd.DataFrame(intraCV2).to_csv('../data/optimized_subsets/AE4intraCV2.csv', index=False)
corr  = corr[corr  <= 0.05].index
pd.DataFrame(corr).to_csv('../data/optimized_subsets/AE4corr.csv', index=False)

In [6]:
ind_CV2 = get_ind_genes(AE3.index, CV2_of_mean)
ind_CV2_genes = np.zeros((len(AE3.index),))
ind_CV2_genes[ind_CV2] = True
ind_CV2_genes = list(ind_CV2_genes.astype(bool))

ind_intraCV2 = get_ind_genes(AE3.index, intraCV2)
ind_intraCV2_genes = np.zeros((len(AE3.index),))
ind_intraCV2_genes[ind_intraCV2] = True
ind_intraCV2_genes = list(ind_intraCV2_genes.astype(bool))

ind_corr = get_ind_genes(AE3.index, corr)
ind_corr_genes = np.zeros((len(AE3.index),))
ind_corr_genes[ind_corr] = True
ind_corr_genes= list(ind_corr_genes.astype(bool))

In [5]:
def get_ind_genes(all_genes:np.array, subset:np.array):
    ind = []
    for gene in subset:
        ind = np.append(ind, int(np.squeeze(np.where(all_genes == gene))))
        ind = list(ind.astype(int))
        
    return ind

In [9]:
subsets = [np.squeeze([ind_CV2_genes]), np.squeeze([ind_intraCV2_genes]), np.squeeze([ind_corr_genes])]
model = EnsemblingHierarchical(np.unique(y_AE3),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = np.array(AE3).T, y= y_AE3)
print(model.score_, model.recovery)

(3, 333)
0.5174129353233831 0.7477477477477478


# Unsupervised genes subset selection

In [22]:
from scipy.cluster.hierarchy import cophenet
import numpy as np

def compute_cophe_coeff(orign_dists:np.array, Z:np.array):
    '''Compute the Cophenetic coefficient from the original distance matrix and generated dendrogram of clustering
        parameters:
        -------
        orign_dists : np.array,
            original distance matrix before clustering
        Z : np.array,
            dendrogram of the clustering to evaluate
    
        Returns
        -------
        corr_coeff:float [0,1],
            computed Cophenetic coefficient '''
    cophe_dists = cophenet(Z) 
    corr_coef = np.corrcoef(orign_dists, cophe_dists)[0,1]
    
    return corr_coef

In [11]:
import numpy as np
import pandas as pd
import sklearn
import math 
from typing import AnyStr, Callable, Tuple
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.base import ClusterMixin,BaseEstimator
from scipy.cluster.hierarchy import ward, cut_tree
from sklearn.metrics import make_scorer
from scipy_cut_tree_balanced import cut_tree_balanced

from pred_score import *

In [15]:
class FamiliesClusters(ClusterMixin, BaseEstimator):
    '''
    Families clustering
    Hierachical clustering with the ward2 criterion, use the spearman correlation as the distance measure.
    Parameters
    ----------
    family_interest: np.array,
        list of family of interest
    Scoring : Callable,
        scoring function use to evaluate the model
    maximize: bool,
        if True the scoring function is maximize, else it is minimize
        
    Attributes
    ----------
    n_clusters_ : int
        The number of clusters found by the algorithm.
    labels_ : ndarray of shape (n_samples)
        Cluster labels for each point.
    family_interest: np.array,
        list of family of interest.
    Scoring_ : Callable,
        scoring function use to evaluate the model.
    maximize_: bool,
        if True the scoring function is maximize, else it is minimize.
    
    '''
    
    def __init__(self, family_interest_:np.array, Scoring_:Callable, maximize_:bool):
        super().__init__()
        self.family_interest_ = family_interest_
        self.Scoring_ = Scoring_
        self.maximize_ = maximize_
        
    def fit(self, X:np.array, y:np.array, NmaxCluster:int = None):
        '''Fit data using hierachical clustering with the ward2 criterion and use the spearman correlation as the distance measure and predict.
        
        parameters:
        -------
        x : np.array,
            features of each data points
        y : np.array,
            family of each data points
        NmaxCluster : int,
            max number of cells in a cluster
        

        Returns
        -------
        self,
            return fitted self'''
        
        #Compute the spearman correlation of X
        X_pd = pd.DataFrame(X.T)
        corr_expr= X_pd.corr(method= 'spearman')
        corr_expr = np.array((1 - corr_expr)/2)
        corr_expr = upper_tri_indexing(corr_expr)**2 #Squared for ward2 criterion
        
        if(np.shape(X.T)[0] == 1):
            corr_expr.fill(1)
        
        #Create clustering tree using hierarchical clustering with spearmann correlation and ward2 criterion
        Z = ward(corr_expr)
        
        #Cut the tree into clusters of maximum size equal to the number of cells in the largest family in data set
        if NmaxCluster == None:
            Nmax = round(np.mean(np.unique(y,return_counts=True)[1]))
        else:
            Nmax = NmaxCluster
        
        clustering = np.squeeze(cut_tree_balanced(Z, max_cluster_size = Nmax)[0])
        
        #Assign all cells predicted alone in a cluster the label 0
        clustering += 1
        values, counts = np.unique(clustering, return_counts=True)
        onecell_family = values[np.where(counts==1)]
        for fam in onecell_family:
            clustering[clustering == fam] = 0
        
        #Compute recovery
        self.recovery = compute_recovery(clustering)
    
        #Score the cluster and determine the number of clusters
        if self.Scoring_ != compute_cophe_coeff:
            score = self.Scoring_(y,clustering)
        else:
            score = self.Scoring_(corr_expr,Z)
            
        N = len(np.unique(clustering))
   
        self.n_clusters_, self.labels_, self.score_ = N, clustering, score
        return self
    
    def fit_predict(self, X:np.array, y:np.array,NmaxCluster:int = None):
        self.fit(X,y,NmaxCluster)
        
        return self.labels_      
    
    def score(self, X, y_true):
        #Error come from here y_true and X not same size as self.labels_ -> function fit_as
        return self.score_

In [16]:
AE3= np.array(pd.read_csv ('../data/processed_data/AE3.csv'))
y = np.array(AE3[:,-1],dtype=int)
AE3 = AE3[:,0:-1]
AE3.shape

(333, 11894)

In [23]:
model = FamiliesClusters(np.unique(y),compute_cophe_coeff,True)
pred = model.fit_predict(AE3,y)
print(model.score_, model.recovery)

0.22874411254286686 0.6726726726726727


In [24]:
genes_AE3 = np.squeeze(pd.read_csv ('../data/processed_data/AE3genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE3genes_bestMIM.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_AE3 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
AE3 = AE3[:,ind_opt_genes]
print(AE3.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../data/optimized_subsets/AE3genes_bestMIM.csv'

In [ ]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE3,y)
print(model.score_, model.recovery)